POISE - Playoff-Oriented Impact and Skill Estimator

Análisis de desempeño y segmentación de jugadores de baloncesto de la NBA mediante técnicas de
Machine Learning.

Estudiante: Daniel Merizalde Restrepo 

Tutor: José Carlos González Dorado

In [1]:
# Importación de librerías:

import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from functools import reduce
import numpy as np
import re

In [2]:
# Definición de urls para descargar datos desde el sitio de la NBA

traditional_per100poss_url = "https://www.nba.com/stats/players/traditional?SeasonType={}&Season={}&PerMode=Per100Possessions"

advanced_per100poss_url = "https://www.nba.com/stats/players/advanced?SeasonType={}&Season={}&PerMode=Per100Possessions"

misc_per100poss_url = "https://www.nba.com/stats/players/misc?SeasonType={}&Season={}&PerMode=Per100Possessions"

scoring_per100poss_url = "https://www.nba.com/stats/players/scoring?SeasonType={}&Season={}&PerMode=Per100Possessions"

defense_per100poss_url = "https://www.nba.com/stats/players/defense?SeasonType={}&Season={}&PerMode=Per100Possessions"

In [3]:
# Definición de urls para descargar datos TRACKING desde el sitio de la NBA

track_drives_url = "https://www.nba.com/stats/players/drives?SeasonType={}&Season={}&PerMode=PerGame"

track_def_impact_url = "https://www.nba.com/stats/players/defensive-impact?SeasonType={}&Season={}&PerMode=PerGame"

track_catchandshoot_url = "https://www.nba.com/stats/players/catch-shoot?SeasonType={}&Season={}&PerMode=PerGame"

track_passing_url = "https://www.nba.com/stats/players/passing?SeasonType={}&Season={}&PerMode=PerGame"

track_pullup_url = "https://www.nba.com/stats/players/pullup?SeasonType={}&Season={}&PerMode=PerGame"

track_reb_url = "https://www.nba.com/stats/players/rebounding?SeasonType={}&Season={}&PerMode=PerGame"

track_offreb_url = "https://www.nba.com/stats/players/offensive-rebounding?SeasonType={}&Season={}&PerMode=PerGame"

track_defreb_url = "https://www.nba.com/stats/players/defensive-rebounding?SeasonType={}&Season={}&PerMode=PerGame"

track_shoot_eff_url = "https://www.nba.com/stats/players/shooting-efficiency?SeasonType={}&Season={}&PerMode=PerGame"

track_speed_url = "https://www.nba.com/stats/players/speed-distance?SeasonType={}&Season={}&PerMode=PerGame"

In [3]:
# Definición de urls para descargar datos OTRAS CATEGORIAS desde el sitio de la NBA

shoot_by_zone_url = "https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&SeasonType={}&Season={}"

# hustle_url = "https://www.nba.com/stats/players/hustle?SeasonType={}&Season={}"

# boxout_url = "https://www.nba.com/stats/players/box-outs?SeasonType={}&Season={}"

In [4]:
# Definición de función que transforma dataframe en archivo csv y lo gusrda en la carpeta de dataset

def dataframe_to_csv(dataframe, folder, csv_file_name):
    path = "csv_data/{}/{}.csv".format(folder, csv_file_name)
    dataframe.to_csv(path, index=False)

In [5]:
""" Definición de función para realizar varias acciones en el sitio
 web de la NBA antes de descargar el archivo html de donde se va a obtener la tabla"""

def scrape_nba_historic_data(selected_url, metric_name):
    
    url = selected_url
    web_data = driver.get(url)
    dropdown1 =  Select(driver.find_element(by=By.XPATH, value = '//*[@id="__next"]/div[2]/div[2]/main/div[2]/section/div/div\
                                                [2]/div[1]/div[7]/div/div[3]/div/label/div/select'))
    
    dropdown2 = driver.find_element(by=By.XPATH, value = '//*[@id="__next"]/div[2]/div[2]/main/div[2]/section/div/div[2]/div\
                                        [1]/div[6]/label/div/span')
    time.sleep(5)
    dropdown2.click() # Hacer click en switch para activar los jugadores históricos en lugar de solamente la última temporada
    time.sleep(5)
    dropdown1.select_by_visible_text("All") # Hacer click en menú dropdown para que muestre todas las filas de la tabla
    time.sleep(15) 
    driver.execute_script("window.scrollTo(1,10000)") # Hacer scroll hasta la parte baja de la página para no perder información
    time.sleep(2)

    html_data = driver.page_source
    
    # Escribir información obtenida por el driver en un archivo .html    
    with open("html_data/{}.html".format(metric_name), "w+", encoding="utf-8") as f:
        f.write(html_data)

In [6]:
""" Definición de función para realizar varias acciones en el sitio
 web de la NBA antes de descargar el archivo html de donde se va a obtener la tabla"""

def scrape_nba_season_data(selected_url, folder_name, metric_name, season, seas_type):
    
    url = selected_url.format(seas_type, season)
    web_data = driver.get(url)
    time.sleep(10)

    dropdown_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select'
    dropdown1 =  Select(driver.find_element(by=By.XPATH, value = dropdown_xpath))
    dropdown1.select_by_visible_text("All") # Hacer click en menú dropdown para que muestre todas las filas de la tabla
    time.sleep(15) 
    
    driver.execute_script("window.scrollTo(1,10000)") # Hacer scroll hasta la parte baja de la página para no perder información
    time.sleep(2)

    html_data = driver.page_source
    
    # Escribir información obtenida por el driver en un archivo .html    
    with open("html_data/{}/{}.html".format(folder_name, metric_name), "w+", encoding="utf-8") as f:
        f.write(html_data)

In [7]:
# Definición de función para hacer "Parsing" en el archivo .html y obtener la tabla deseada

def parse_html(folder_name, file_name, id_name, decompose_id, decompose_class):
    with open("html_data/{}/{}.html".format(folder_name, file_name), encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    # soup.find(decompose_id, class_=decompose_class).decompose()
    # data_table = soup.find(id=str(id_name))
    data_table = soup.find(class_=str(id_name))
    dataframe = pd.read_html(str(data_table), displayed_only=False)[0]
    
    return dataframe

In [12]:
# Inicialización de driver Chrome para hacer Webscraping

s = Service("D:/Users/user/Desktop/DANI/Programación y DS/Drivers/chromedriver.exe")
driver = webdriver.Chrome(service=s)

In [9]:
seasons_list = ["2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", \
               "2019-20", "2020-21", "2021-22", "2022-23"]

In [10]:
increased_season_ls = []

for year in range(1996, 2023):
    season = f"{year}-{str(year + 1)[-2:]}"
    increased_season_ls.append(season)

print(increased_season_ls)

['1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']


In [15]:
# Función FOR para descarga de archivos estadísticas tradicionales por 100 posesiones en TEMPORADA REGULAR

for seas in increased_season_ls:
    filename = "regseas_trad_per100_{}".format(seas)
    scrape_nba_season_data(traditional_per100poss_url, folder_name = 'traditional', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'traditional', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'traditional', filename)
    print("saved: ", filename)

saved:  regseas_trad_per100_1996-97
saved:  regseas_trad_per100_1997-98
saved:  regseas_trad_per100_1998-99
saved:  regseas_trad_per100_1999-00
saved:  regseas_trad_per100_2000-01
saved:  regseas_trad_per100_2001-02
saved:  regseas_trad_per100_2002-03
saved:  regseas_trad_per100_2003-04
saved:  regseas_trad_per100_2004-05
saved:  regseas_trad_per100_2005-06
saved:  regseas_trad_per100_2006-07
saved:  regseas_trad_per100_2007-08
saved:  regseas_trad_per100_2008-09
saved:  regseas_trad_per100_2009-10
saved:  regseas_trad_per100_2010-11
saved:  regseas_trad_per100_2011-12
saved:  regseas_trad_per100_2012-13
saved:  regseas_trad_per100_2013-14
saved:  regseas_trad_per100_2014-15
saved:  regseas_trad_per100_2015-16
saved:  regseas_trad_per100_2016-17
saved:  regseas_trad_per100_2017-18
saved:  regseas_trad_per100_2018-19


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"}
  (Session info: chrome=120.0.6099.71)
Stacktrace:
	GetHandleVerifier [0x00007FF6E5304D02+56194]
	(No symbol) [0x00007FF6E52704B2]
	(No symbol) [0x00007FF6E51176AA]
	(No symbol) [0x00007FF6E51616D0]
	(No symbol) [0x00007FF6E51617EC]
	(No symbol) [0x00007FF6E51A4D77]
	(No symbol) [0x00007FF6E5185EBF]
	(No symbol) [0x00007FF6E51A2786]
	(No symbol) [0x00007FF6E5185C23]
	(No symbol) [0x00007FF6E5154A45]
	(No symbol) [0x00007FF6E5155AD4]
	GetHandleVerifier [0x00007FF6E567D5BB+3695675]
	GetHandleVerifier [0x00007FF6E56D6197+4059159]
	GetHandleVerifier [0x00007FF6E56CDF63+4025827]
	GetHandleVerifier [0x00007FF6E539F029+687785]
	(No symbol) [0x00007FF6E527B508]
	(No symbol) [0x00007FF6E5277564]
	(No symbol) [0x00007FF6E52776E9]
	(No symbol) [0x00007FF6E5268094]
	BaseThreadInitThunk [0x0

In [ ]:
# Función FOR para descarga de archivos estadísticas tradicionales por 100 posesiones en TEMPORADA REGULAR

for seas in increased_season_ls:
    filename = "playoff_trad_per100_{}".format(seas)
    scrape_nba_season_data(traditional_per100poss_url, folder_name = 'traditional', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'traditional', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'traditional', filename)
    print("saved: ", filename)

In [18]:
# Función FOR para descarga de archivos estadísticas avanzadas por 100 posesiones en TEMPORADA REGULAR y PLAYOFFS

for seas in increased_season_ls:
    filename = "regseas_adv_per100_{}".format(seas)
    scrape_nba_season_data(advanced_per100poss_url, folder_name = 'advanced', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'advanced', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'advanced', filename)
    print("saved: ", filename)
    
'''    
# Playoffs

for seas in seasons_list:
    filename = "playoff_adv_per100_{}".format(seas)
    scrape_nba_season_data(advanced_per100poss_url, folder_name = 'advanced', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'advanced', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'advanced', filename)
    print("saved: ", filename)
'''

saved:  regseas_adv_per100_1996-97
saved:  regseas_adv_per100_1997-98
saved:  regseas_adv_per100_1998-99
saved:  regseas_adv_per100_1999-00
saved:  regseas_adv_per100_2000-01
saved:  regseas_adv_per100_2001-02
saved:  regseas_adv_per100_2002-03
saved:  regseas_adv_per100_2003-04
saved:  regseas_adv_per100_2004-05
saved:  regseas_adv_per100_2005-06
saved:  regseas_adv_per100_2006-07
saved:  regseas_adv_per100_2007-08
saved:  regseas_adv_per100_2008-09
saved:  regseas_adv_per100_2009-10
saved:  regseas_adv_per100_2010-11
saved:  regseas_adv_per100_2011-12
saved:  regseas_adv_per100_2012-13
saved:  regseas_adv_per100_2013-14
saved:  regseas_adv_per100_2014-15
saved:  regseas_adv_per100_2015-16
saved:  regseas_adv_per100_2016-17
saved:  regseas_adv_per100_2017-18
saved:  regseas_adv_per100_2018-19
saved:  regseas_adv_per100_2019-20
saved:  regseas_adv_per100_2020-21
saved:  regseas_adv_per100_2021-22
saved:  regseas_adv_per100_2022-23


'    \n# Playoffs\n\nfor seas in seasons_list:\n    filename = "playoff_adv_per100_{}".format(seas)\n    scrape_nba_season_data(advanced_per100poss_url, folder_name = \'advanced\',                            metric_name = filename, season = seas, seas_type = \'Playoffs\')\n    time.sleep(1)\n    dataframe = parse_html(folder_name = \'advanced\', file_name = filename, id_name="Crom_table__p1iZz",                            decompose_id=\'tr\', decompose_class="thead")\n    dataframe_to_csv(dataframe, \'advanced\', filename)\n    print("saved: ", filename)\n'

In [ ]:
# Función FOR para descarga de archivos estadísticas miscelanea por 100 posesiones en TEMPORADA REGULAR y PLAYOFFS

for seas in increased_season_ls:
    filename = "regseas_misc_per100_{}".format(seas)
    scrape_nba_season_data(misc_per100poss_url, folder_name = 'miscellaneous', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'miscellaneous', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'miscellaneous', filename)
    print("saved: ", filename)
    
# Playoffs
'''
for seas in seasons_list:
    filename = "playoff_misc_per100_{}".format(seas)
    scrape_nba_season_data(misc_per100poss_url, folder_name = 'miscellaneous', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'miscellaneous', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'miscellaneous', filename)
    print("saved: ", filename)
'''

saved:  regseas_misc_per100_1996-97
saved:  regseas_misc_per100_1997-98
saved:  regseas_misc_per100_1998-99
saved:  regseas_misc_per100_1999-00
saved:  regseas_misc_per100_2000-01
saved:  regseas_misc_per100_2001-02
saved:  regseas_misc_per100_2002-03
saved:  regseas_misc_per100_2003-04
saved:  regseas_misc_per100_2004-05
saved:  regseas_misc_per100_2005-06
saved:  regseas_misc_per100_2006-07
saved:  regseas_misc_per100_2007-08
saved:  regseas_misc_per100_2008-09
saved:  regseas_misc_per100_2009-10
saved:  regseas_misc_per100_2010-11
saved:  regseas_misc_per100_2011-12
saved:  regseas_misc_per100_2012-13
saved:  regseas_misc_per100_2013-14
saved:  regseas_misc_per100_2014-15
saved:  regseas_misc_per100_2015-16


In [13]:
# Función FOR para descarga de archivos estadísticas scoring por 100 posesiones en TEMPORADA REGULAR y PLAYOFFS

for seas in increased_season_ls:
    filename = "regseas_scoring_per100_{}".format(seas)
    scrape_nba_season_data(scoring_per100poss_url, folder_name = 'scoring', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'scoring', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'scoring', filename)
    print("saved: ", filename)
    
# Playoffs
'''
for seas in seasons_list:
    filename = "playoff_scoring_per100_{}".format(seas)
    scrape_nba_season_data(scoring_per100poss_url, folder_name = 'scoring', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'scoring', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'scoring', filename)
    print("saved: ", filename)
'''

saved:  regseas_scoring_per100_1996-97
saved:  regseas_scoring_per100_1997-98
saved:  regseas_scoring_per100_1998-99
saved:  regseas_scoring_per100_1999-00
saved:  regseas_scoring_per100_2000-01
saved:  regseas_scoring_per100_2001-02
saved:  regseas_scoring_per100_2002-03
saved:  regseas_scoring_per100_2003-04
saved:  regseas_scoring_per100_2004-05
saved:  regseas_scoring_per100_2005-06
saved:  regseas_scoring_per100_2006-07
saved:  regseas_scoring_per100_2007-08
saved:  regseas_scoring_per100_2008-09
saved:  regseas_scoring_per100_2009-10
saved:  regseas_scoring_per100_2010-11
saved:  regseas_scoring_per100_2011-12
saved:  regseas_scoring_per100_2012-13
saved:  regseas_scoring_per100_2013-14


KeyboardInterrupt: 

In [15]:
# Función FOR para descarga de archivos estadísticas defense por 100 posesiones en TEMPORADA REGULAR y PLAYOFFS

for seas in increased_season_ls:
    filename = "regseas_defense_per100_{}".format(seas)
    scrape_nba_season_data(defense_per100poss_url, folder_name = 'defense', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'defense', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'defense', filename)
    print("saved: ", filename)
    
# Playoffs
'''
for seas in seasons_list:
    filename = "playoff_defense_per100_{}".format(seas)
    scrape_nba_season_data(defense_per100poss_url, folder_name = 'defense', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'defense', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'defense', filename)
    print("saved: ", filename)
'''

saved:  regseas_defense_per100_1996-97
saved:  regseas_defense_per100_1997-98
saved:  regseas_defense_per100_1998-99
saved:  regseas_defense_per100_1999-00
saved:  regseas_defense_per100_2000-01
saved:  regseas_defense_per100_2001-02
saved:  regseas_defense_per100_2002-03
saved:  regseas_defense_per100_2003-04
saved:  regseas_defense_per100_2004-05
saved:  regseas_defense_per100_2005-06
saved:  regseas_defense_per100_2006-07
saved:  regseas_defense_per100_2007-08
saved:  regseas_defense_per100_2008-09
saved:  regseas_defense_per100_2009-10
saved:  regseas_defense_per100_2010-11
saved:  regseas_defense_per100_2011-12
saved:  regseas_defense_per100_2012-13
saved:  regseas_defense_per100_2013-14
saved:  regseas_defense_per100_2014-15
saved:  regseas_defense_per100_2015-16
saved:  regseas_defense_per100_2016-17
saved:  regseas_defense_per100_2017-18


KeyboardInterrupt: 

In [16]:
# Función FOR para descarga de archivos estadísticas drives por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in increased_season_ls:
    filename = "regseas_drives_pergame_{}".format(seas)
    scrape_nba_season_data(track_drives_url, folder_name = 'track_drives', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_drives', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_drives', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_drives_pergame_{}".format(seas)
    scrape_nba_season_data(track_drives_url, folder_name = 'track_drives', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_drives', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_drives', filename)
    print("saved: ", filename)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"}
  (Session info: chrome=120.0.6099.72)
Stacktrace:
	GetHandleVerifier [0x00007FF6E2984D02+56194]
	(No symbol) [0x00007FF6E28F04B2]
	(No symbol) [0x00007FF6E27976AA]
	(No symbol) [0x00007FF6E27E16D0]
	(No symbol) [0x00007FF6E27E17EC]
	(No symbol) [0x00007FF6E2824D77]
	(No symbol) [0x00007FF6E2805EBF]
	(No symbol) [0x00007FF6E2822786]
	(No symbol) [0x00007FF6E2805C23]
	(No symbol) [0x00007FF6E27D4A45]
	(No symbol) [0x00007FF6E27D5AD4]
	GetHandleVerifier [0x00007FF6E2CFD5BB+3695675]
	GetHandleVerifier [0x00007FF6E2D56197+4059159]
	GetHandleVerifier [0x00007FF6E2D4DF63+4025827]
	GetHandleVerifier [0x00007FF6E2A1F029+687785]
	(No symbol) [0x00007FF6E28FB508]
	(No symbol) [0x00007FF6E28F7564]
	(No symbol) [0x00007FF6E28F76E9]
	(No symbol) [0x00007FF6E28E8094]
	BaseThreadInitThunk [0x0

In [13]:
# Función FOR para descarga de archivos estadísticas def. Impact por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in seasons_list:
    filename = "regseas_def_impact_pergame_{}".format(seas)
    scrape_nba_season_data(track_def_impact_url, folder_name = 'track_def_impact', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_def_impact', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_def_impact', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_def_impact_pergame_{}".format(seas)
    scrape_nba_season_data(track_def_impact_url, folder_name = 'track_def_impact', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_def_impact', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_def_impact', filename)
    print("saved: ", filename)

saved:  regseas_def_impact_pergame_2013-14
saved:  regseas_def_impact_pergame_2014-15
saved:  regseas_def_impact_pergame_2015-16
saved:  regseas_def_impact_pergame_2016-17
saved:  regseas_def_impact_pergame_2017-18
saved:  regseas_def_impact_pergame_2018-19
saved:  regseas_def_impact_pergame_2019-20
saved:  regseas_def_impact_pergame_2020-21
saved:  regseas_def_impact_pergame_2021-22
saved:  regseas_def_impact_pergame_2022-23
saved:  playoff_def_impact_pergame_2013-14
saved:  playoff_def_impact_pergame_2014-15
saved:  playoff_def_impact_pergame_2015-16
saved:  playoff_def_impact_pergame_2016-17
saved:  playoff_def_impact_pergame_2017-18
saved:  playoff_def_impact_pergame_2018-19
saved:  playoff_def_impact_pergame_2019-20
saved:  playoff_def_impact_pergame_2020-21
saved:  playoff_def_impact_pergame_2021-22
saved:  playoff_def_impact_pergame_2022-23


In [21]:
# Función FOR para descarga de archivos estadísticas catch and shoot por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in seasons_list:
    filename = "regseas_catchandshoot_pergame_{}".format(seas)
    scrape_nba_season_data(track_catchandshoot_url, folder_name = 'track_catch_and_shoot', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_catch_and_shoot', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_catch_and_shoot', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_catchandshoot_pergame_{}".format(seas)
    scrape_nba_season_data(track_catchandshoot_url, folder_name = 'track_catch_and_shoot', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_catch_and_shoot', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_catch_and_shoot', filename)
    print("saved: ", filename)

saved:  regseas_catchandshoot_pergame_2013-14
saved:  regseas_catchandshoot_pergame_2014-15
saved:  regseas_catchandshoot_pergame_2015-16
saved:  regseas_catchandshoot_pergame_2016-17
saved:  regseas_catchandshoot_pergame_2017-18
saved:  regseas_catchandshoot_pergame_2018-19
saved:  regseas_catchandshoot_pergame_2019-20
saved:  regseas_catchandshoot_pergame_2020-21
saved:  regseas_catchandshoot_pergame_2021-22
saved:  regseas_catchandshoot_pergame_2022-23
saved:  playoff_catchandshoot_pergame_2013-14
saved:  playoff_catchandshoot_pergame_2014-15
saved:  playoff_catchandshoot_pergame_2015-16
saved:  playoff_catchandshoot_pergame_2016-17
saved:  playoff_catchandshoot_pergame_2017-18
saved:  playoff_catchandshoot_pergame_2018-19
saved:  playoff_catchandshoot_pergame_2019-20
saved:  playoff_catchandshoot_pergame_2020-21
saved:  playoff_catchandshoot_pergame_2021-22
saved:  playoff_catchandshoot_pergame_2022-23


In [22]:
# Función FOR para descarga de archivos estadísticas catch and shoot por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in seasons_list:
    filename = "regseas_passing_pergame_{}".format(seas)
    scrape_nba_season_data(track_passing_url, folder_name = 'track_passing', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_passing', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_passing', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_passing_pergame_{}".format(seas)
    scrape_nba_season_data(track_passing_url, folder_name = 'track_passing', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_passing', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_passing', filename)
    print("saved: ", filename)

saved:  regseas_passing_pergame_2013-14
saved:  regseas_passing_pergame_2014-15
saved:  regseas_passing_pergame_2015-16
saved:  regseas_passing_pergame_2016-17
saved:  regseas_passing_pergame_2017-18
saved:  regseas_passing_pergame_2018-19
saved:  regseas_passing_pergame_2019-20
saved:  regseas_passing_pergame_2020-21
saved:  regseas_passing_pergame_2021-22
saved:  regseas_passing_pergame_2022-23
saved:  playoff_passing_pergame_2013-14
saved:  playoff_passing_pergame_2014-15
saved:  playoff_passing_pergame_2015-16
saved:  playoff_passing_pergame_2016-17
saved:  playoff_passing_pergame_2017-18
saved:  playoff_passing_pergame_2018-19
saved:  playoff_passing_pergame_2019-20
saved:  playoff_passing_pergame_2020-21
saved:  playoff_passing_pergame_2021-22
saved:  playoff_passing_pergame_2022-23


In [30]:
# Función FOR para descarga de archivos estadísticas catch and shoot por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in seasons_list:
    filename = "regseas_pullup_pergame_{}".format(seas)
    scrape_nba_season_data(track_pullup_url, folder_name = 'track_pull_up_shoot', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_pull_up_shoot', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_pull_up_shoot', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_pullup_pergame_{}".format(seas)
    scrape_nba_season_data(track_pullup_url, folder_name = 'track_pull_up_shoot', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_pull_up_shoot', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_pull_up_shoot', filename)
    print("saved: ", filename)

saved:  regseas_pullup_pergame_2013-14
saved:  regseas_pullup_pergame_2014-15
saved:  regseas_pullup_pergame_2015-16
saved:  regseas_pullup_pergame_2016-17
saved:  regseas_pullup_pergame_2017-18
saved:  regseas_pullup_pergame_2018-19
saved:  regseas_pullup_pergame_2019-20
saved:  regseas_pullup_pergame_2020-21
saved:  regseas_pullup_pergame_2021-22
saved:  regseas_pullup_pergame_2022-23
saved:  playoff_pullup_pergame_2013-14
saved:  playoff_pullup_pergame_2014-15
saved:  playoff_pullup_pergame_2015-16
saved:  playoff_pullup_pergame_2016-17
saved:  playoff_pullup_pergame_2017-18
saved:  playoff_pullup_pergame_2018-19
saved:  playoff_pullup_pergame_2019-20
saved:  playoff_pullup_pergame_2020-21
saved:  playoff_pullup_pergame_2021-22
saved:  playoff_pullup_pergame_2022-23


In [25]:
# Función FOR para descarga de archivos estadísticas rebounding por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in seasons_list:
    filename = "regseason_rebounding_pergame_{}".format(seas)
    scrape_nba_season_data(track_reb_url, folder_name = 'track_rebounding', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_rebounding', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_rebounding', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_rebounding_pergame_{}".format(seas)
    scrape_nba_season_data(track_reb_url, folder_name = 'track_rebounding', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_rebounding', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_rebounding', filename)
    print("saved: ", filename)

saved:  regseason_rebounding_pergame_2013-14
saved:  regseason_rebounding_pergame_2014-15
saved:  regseason_rebounding_pergame_2015-16
saved:  regseason_rebounding_pergame_2016-17
saved:  regseason_rebounding_pergame_2017-18
saved:  regseason_rebounding_pergame_2018-19
saved:  regseason_rebounding_pergame_2019-20
saved:  regseason_rebounding_pergame_2020-21
saved:  regseason_rebounding_pergame_2021-22
saved:  regseason_rebounding_pergame_2022-23
saved:  playoff_rebounding_pergame_2013-14
saved:  playoff_rebounding_pergame_2014-15
saved:  playoff_rebounding_pergame_2015-16
saved:  playoff_rebounding_pergame_2016-17
saved:  playoff_rebounding_pergame_2017-18
saved:  playoff_rebounding_pergame_2018-19
saved:  playoff_rebounding_pergame_2019-20
saved:  playoff_rebounding_pergame_2020-21
saved:  playoff_rebounding_pergame_2021-22
saved:  playoff_rebounding_pergame_2022-23


In [26]:
# Función FOR para descarga de archivos estadísticas off rebounding por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in seasons_list:
    filename = "regseas_offreb_pergame_{}".format(seas)
    scrape_nba_season_data(track_offreb_url, folder_name = 'track_off_reb', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_off_reb', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_off_reb', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_offreb_pergame_{}".format(seas)
    scrape_nba_season_data(track_offreb_url, folder_name = 'track_off_reb', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_off_reb', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_off_reb', filename)
    print("saved: ", filename)

saved:  regseas_offreb_pergame_2013-14
saved:  regseas_offreb_pergame_2014-15
saved:  regseas_offreb_pergame_2015-16
saved:  regseas_offreb_pergame_2016-17
saved:  regseas_offreb_pergame_2017-18
saved:  regseas_offreb_pergame_2018-19
saved:  regseas_offreb_pergame_2019-20
saved:  regseas_offreb_pergame_2020-21
saved:  regseas_offreb_pergame_2021-22
saved:  regseas_offreb_pergame_2022-23
saved:  playoff_offreb_pergame_2013-14
saved:  playoff_offreb_pergame_2014-15
saved:  playoff_offreb_pergame_2015-16
saved:  playoff_offreb_pergame_2016-17
saved:  playoff_offreb_pergame_2017-18
saved:  playoff_offreb_pergame_2018-19
saved:  playoff_offreb_pergame_2019-20
saved:  playoff_offreb_pergame_2020-21
saved:  playoff_offreb_pergame_2021-22
saved:  playoff_offreb_pergame_2022-23


In [31]:
# Función FOR para descarga de archivos estadísticas Def rebounding por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in seasons_list:
    filename = "regseas_defreb_pergame_{}".format(seas)
    scrape_nba_season_data(track_defreb_url, folder_name = 'track_def_reb', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_def_reb', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_def_reb', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_defreb_pergame_{}".format(seas)
    scrape_nba_season_data(track_defreb_url, folder_name = 'track_def_reb', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_def_reb', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_def_reb', filename)
    print("saved: ", filename)

saved:  regseas_defreb_pergame_2013-14
saved:  regseas_defreb_pergame_2014-15
saved:  regseas_defreb_pergame_2015-16
saved:  regseas_defreb_pergame_2016-17
saved:  regseas_defreb_pergame_2017-18
saved:  regseas_defreb_pergame_2018-19
saved:  regseas_defreb_pergame_2019-20
saved:  regseas_defreb_pergame_2020-21
saved:  regseas_defreb_pergame_2021-22
saved:  regseas_defreb_pergame_2022-23
saved:  playoff_defreb_pergame_2013-14
saved:  playoff_defreb_pergame_2014-15
saved:  playoff_defreb_pergame_2015-16
saved:  playoff_defreb_pergame_2016-17
saved:  playoff_defreb_pergame_2017-18
saved:  playoff_defreb_pergame_2018-19
saved:  playoff_defreb_pergame_2019-20
saved:  playoff_defreb_pergame_2020-21
saved:  playoff_defreb_pergame_2021-22
saved:  playoff_defreb_pergame_2022-23


In [11]:
# Función FOR para descarga de archivos estadísticas shooting efficiency por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in seasons_list:
    filename = "regseas_shoot_eff_pergame_{}".format(seas)
    scrape_nba_season_data(track_shoot_eff_url, folder_name = 'track_shoot_eff', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_shoot_eff', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_shoot_eff', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_shoot_eff_pergame_{}".format(seas)
    scrape_nba_season_data(track_shoot_eff_url, folder_name = 'track_shoot_eff', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_shoot_eff', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_shoot_eff', filename)
    print("saved: ", filename)

saved:  regseas_shoot_eff_pergame_2013-14
saved:  regseas_shoot_eff_pergame_2014-15
saved:  regseas_shoot_eff_pergame_2015-16
saved:  regseas_shoot_eff_pergame_2016-17
saved:  regseas_shoot_eff_pergame_2017-18
saved:  regseas_shoot_eff_pergame_2018-19
saved:  regseas_shoot_eff_pergame_2019-20
saved:  regseas_shoot_eff_pergame_2020-21
saved:  regseas_shoot_eff_pergame_2021-22
saved:  regseas_shoot_eff_pergame_2022-23
saved:  playoff_shoot_eff_pergame_2013-14
saved:  playoff_shoot_eff_pergame_2014-15
saved:  playoff_shoot_eff_pergame_2015-16
saved:  playoff_shoot_eff_pergame_2016-17
saved:  playoff_shoot_eff_pergame_2017-18
saved:  playoff_shoot_eff_pergame_2018-19
saved:  playoff_shoot_eff_pergame_2019-20
saved:  playoff_shoot_eff_pergame_2020-21
saved:  playoff_shoot_eff_pergame_2021-22
saved:  playoff_shoot_eff_pergame_2022-23


In [33]:
# Función FOR para descarga de archivos estadísticas speed por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in seasons_list:
    filename = "regseas_speed_pergame_{}".format(seas)
    scrape_nba_season_data(track_speed_url, folder_name = 'track_speed_and_distance', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_speed_and_distance', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_speed_and_distance', filename)
    print("saved: ", filename)
    
# Playoffs

for seas in seasons_list:
    filename = "playoff_speed_pergame_{}".format(seas)
    scrape_nba_season_data(track_speed_url, folder_name = 'track_speed_and_distance', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'track_speed_and_distance', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'track_speed_and_distance', filename)
    print("saved: ", filename)

saved:  regseas_speed_pergame_2013-14
saved:  regseas_speed_pergame_2014-15
saved:  regseas_speed_pergame_2015-16
saved:  regseas_speed_pergame_2016-17
saved:  regseas_speed_pergame_2017-18
saved:  regseas_speed_pergame_2018-19
saved:  regseas_speed_pergame_2019-20
saved:  regseas_speed_pergame_2020-21
saved:  regseas_speed_pergame_2021-22
saved:  regseas_speed_pergame_2022-23
saved:  playoff_speed_pergame_2013-14
saved:  playoff_speed_pergame_2014-15
saved:  playoff_speed_pergame_2015-16
saved:  playoff_speed_pergame_2016-17
saved:  playoff_speed_pergame_2017-18
saved:  playoff_speed_pergame_2018-19
saved:  playoff_speed_pergame_2019-20
saved:  playoff_speed_pergame_2020-21
saved:  playoff_speed_pergame_2021-22
saved:  playoff_speed_pergame_2022-23


In [17]:
# Función FOR para descarga de archivos estadísticas shooting by zone por partido en TEMPORADA REGULAR y PLAYOFFS

for seas in increased_season_ls:
    filename = "regseas_shootzone_pergame_{}".format(seas)
    scrape_nba_season_data(shoot_by_zone_url, folder_name = 'shooting_by_zone', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'shooting_by_zone', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'shooting_by_zone', filename)
    print("saved: ", filename)
    
# Playoffs
'''
for seas in seasons_list:
    filename = "playoff_shootzone_pergame_{}".format(seas)
    scrape_nba_season_data(shoot_by_zone_url, folder_name = 'shooting_by_zone', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'shooting_by_zone', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'shooting_by_zone', filename)
    print("saved: ", filename)
'''

saved:  regseas_shootzone_pergame_1996-97
saved:  regseas_shootzone_pergame_1997-98
saved:  regseas_shootzone_pergame_1998-99
saved:  regseas_shootzone_pergame_1999-00
saved:  regseas_shootzone_pergame_2000-01
saved:  regseas_shootzone_pergame_2001-02
saved:  regseas_shootzone_pergame_2002-03
saved:  regseas_shootzone_pergame_2003-04
saved:  regseas_shootzone_pergame_2004-05
saved:  regseas_shootzone_pergame_2005-06
saved:  regseas_shootzone_pergame_2006-07
saved:  regseas_shootzone_pergame_2007-08
saved:  regseas_shootzone_pergame_2008-09
saved:  regseas_shootzone_pergame_2009-10
saved:  regseas_shootzone_pergame_2010-11
saved:  regseas_shootzone_pergame_2011-12
saved:  regseas_shootzone_pergame_2012-13
saved:  regseas_shootzone_pergame_2013-14
saved:  regseas_shootzone_pergame_2014-15
saved:  regseas_shootzone_pergame_2015-16
saved:  regseas_shootzone_pergame_2016-17
saved:  regseas_shootzone_pergame_2017-18
saved:  regseas_shootzone_pergame_2018-19
saved:  regseas_shootzone_pergame_

KeyboardInterrupt: 

In [13]:
test_shzone_df = parse_html(folder_name = 'shooting_by_zone', file_name = "regseas_shootzone_pergame_2015-16",\
                       id_name="Crom_table__p1iZz", decompose_id='tr', decompose_class="thead")

In [15]:
test_shzone_df.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
              Player               Team                Age                FGM   
0       Aaron Brooks                CHI                 31                0.8   
1       Aaron Gordon                ORL                 20                2.3   
2     Aaron Harrison                CHA                 21                0.1   
3      Adreian Payne                MIN                 25                0.4   
4         Al Horford                ATL                 30                3.0   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Restricted Area             \
                 FGA                FG%             FGM  FGA   FG%   
0                1.8               47.2             0.5  1.2  40.5   
1                3.5               64.7             0.3  0.8  37.5   
2                0.2               40.0             0.0  0.0   0.0   
3                0.8               51.3             0.2  0.6  27.6   
4                4.1               72.5             0.6  1.5  40.3   

  In The Paint(Non-RA)  ... Left Corner 3. Right Corner 3.             \
                   FGM  ...            FG%             FGM  FGA   FG%   
0                  0.4  ...           50.0             0.1  0.3  37.5   
1                  0.4  ...           43.5             0.2  0.7  33.3   
2                  0.0  ...            100             0.1  0.3  25.0   
3                  0.3  ...            0.0             0.1  0.2  25.0   
4                  1.8  ...           25.6             0.2  0.7  26.3   

  Corner 3             Above the Break 3.                      \
       FGM  FGA   FG% Unnamed: 24_level_1 Unnamed: 25_level_1   
0      0.9  2.4  36.4                 NaN                 NaN   
1      0.3  1.0  29.1                 NaN                 NaN   
2      0.1  0.3  33.3                 NaN                 NaN   
3      0.1  0.5  29.2                 NaN                 NaN   
4      0.9  2.4  36.7                 NaN                 NaN   

                       
  Unnamed: 26_level_1  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  

[5 rows x 27 columns]

In [19]:
test_shzone_df.columns = test_shzone_df.columns.droplevel() 

In [21]:
test_shzone_df.head()

,Player,Team,Age,FGM,FGA,FG%,FGM,FGA,FG%,FGM,...,FG%,FGM,FGA,FG%,FGM,FGA,FG%,Unnamed: 24_level_1,Unnamed: 25_level_1,Unnamed: 26_level_1
0,Aaron Brooks,CHI,31,0.8,1.8,47.2,0.5,1.2,40.5,0.4,...,50.0,0.1,0.3,37.5,0.9,2.4,36.4,NaN,NaN,NaN
1,Aaron Gordon,ORL,20,2.3,3.5,64.7,0.3,0.8,37.5,0.4,...,43.5,0.2,0.7,33.3,0.3,1.0,29.1,NaN,NaN,NaN
2,Aaron Harrison,CHA,21,0.1,0.2,40.0,0.0,0.0,0.0,0.0,...,100,0.1,0.3,25.0,0.1,0.3,33.3,NaN,NaN,NaN
3,Adreian Payne,MIN,25,0.4,0.8,51.3,0.2,0.6,27.6,0.3,...,0.0,0.1,0.2,25.0,0.1,0.5,29.2,NaN,NaN,NaN
4,Al Horford,ATL,30,3.0,4.1,72.5,0.6,1.5,40.3,1.8,...,25.6,0.2,0.7,26.3,0.9,2.4,36.7,NaN,NaN,NaN


In [30]:
test_shzone_df.columns

Index(['Player', 'Team', 'Age', 'FGM', 'FGA', 'FG%', 'FGM', 'FGA', 'FG%',
       'FGM', 'FGA', 'FG%', 'FGM', 'FGA', 'FG%', 'FGM', 'FGA', 'FG%', 'FGM',
       'FGA', 'FG%', 'FGM', 'FGA', 'FG%'],
      dtype='object')

In [32]:
shzone_columns = ['Player', 'Team', 'Age', 'RA_FGM', 'RA_FGA', 'RA_FG%', 'PAINT_FGM', 'PAINT_FGA', 'PAINT_FG%',\
                   'MR_FGM', 'MR_FGA', 'MR_FG%', 'LC3_FGM', 'LC3_FGA', 'LC3_FG%', 'RC3_FGM', 'RC3_FGA', 'RC3_FG%', \
                  'C3_FGA', 'C3_FGA', 'C3_FGA', 'AB3_FGM', 'AB3_FGA', 'AB3_FG%']

In [27]:
drop_columns = ['Unnamed: 24_level_1', 'Unnamed: 25_level_1', 'Unnamed: 26_level_1']

In [29]:
test_shzone_df = test_shzone_df.drop(drop_columns, axis=1)

In [33]:
test_shzone_df.columns = shzone_columns

In [36]:
test_shzone_df.sort_values(by="AB3_FGA", ascending=False)

,Player,Team,Age,RA_FGM,RA_FGA,RA_FG%,PAINT_FGM,PAINT_FGA,PAINT_FG%,MR_FGM,...,LC3_FG%,RC3_FGM,RC3_FGA,RC3_FG%,C3_FGA,C3_FGA,C3_FGA,AB3_FGM,AB3_FGA,AB3_FG%
421,Stephen Curry,GSW,28,3.3,5.1,65.9,0.7,1.4,48.7,1.1,...,47.6,0.3,0.7,50.9,0.7,1.5,49.1,4.3,9.6,45.2
95,Damian Lillard,POR,25,3.0,5.7,51.9,0.6,1.9,29.7,1.6,...,34.8,0.1,0.2,42.9,0.2,0.5,37.8,2.9,7.6,37.8
189,James Harden,HOU,26,3.4,5.6,60.7,0.8,2.3,36.7,1.6,...,28.6,0.1,0.5,27.9,0.2,0.8,28.1,2.7,7.2,36.8
264,Kevin Durant,OKC,27,3.1,4.3,70.7,1.3,2.7,49.7,2.7,...,25.0,0.1,0.2,41.2,0.1,0.3,38.1,2.5,6.4,38.7
280,Kyle Lowry,TOR,30,2.3,4.1,55.8,0.6,1.7,36.1,1.0,...,43.9,0.1,0.2,55.6,0.4,0.8,47.5,2.4,6.3,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,Kendrick Perkins,NOP,31,0.5,0.8,65.5,0.5,1.0,48.6,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
261,Kenneth Faried,DEN,26,4.2,6.8,61.5,0.7,1.6,44.9,0.3,...,0.0,0.0,0.0,100,0.0,0.0,100,0.0,0.0,0.0
265,Kevin Garnett,MIN,40,0.3,0.5,63.2,0.3,0.5,55.6,0.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
268,Kevin Seraphin,NYK,26,0.6,1.1,55.8,0.5,1.3,36.1,0.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Función FOR shooting by zone por partido en TEMPORADA REGULAR y PLAYOFFS con LIMPIEZA DE COLUMNAS

for seas in increased_season_ls:
    filename = "regseas_shootzone_pergame_{}".format(seas)
    scrape_nba_season_data(shoot_by_zone_url, folder_name = 'shooting_by_zone', \
                           metric_name = filename, season = seas, seas_type = 'Regular+Season')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'shooting_by_zone', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    
    dataframe.columns = dataframe.columns.droplevel() 
    
    shzone_columns = ['Player', 'Team', 'Age', 'RA_FGM', 'RA_FGA', 'RA_FG%', 'PAINT_FGM', 'PAINT_FGA', 'PAINT_FG%',\
                   'MR_FGM', 'MR_FGA', 'MR_FG%', 'LC3_FGM', 'LC3_FGA', 'LC3_FG%', 'RC3_FGM', 'RC3_FGA', 'RC3_FG%', \
                  'C3_FGA', 'C3_FGA', 'C3_FGA', 'AB3_FGM', 'AB3_FGA', 'AB3_FG%']
    
    drop_columns = ['Unnamed: 24_level_1', 'Unnamed: 25_level_1', 'Unnamed: 26_level_1']
    
    dataframe = dataframe.drop(drop_columns, axis=1)
    dataframe.columns = shzone_columns
    
    dataframe_to_csv(dataframe, 'shooting_by_zone', filename)
    print("saved: ", filename)
    
# Playoffs
'''
for seas in seasons_list:
    filename = "playoff_shootzone_pergame_{}".format(seas)
    scrape_nba_season_data(shoot_by_zone_url, folder_name = 'shooting_by_zone', \
                           metric_name = filename, season = seas, seas_type = 'Playoffs')
    time.sleep(1)
    dataframe = parse_html(folder_name = 'shooting_by_zone', file_name = filename, id_name="Crom_table__p1iZz", \
                           decompose_id='tr', decompose_class="thead")
    dataframe_to_csv(dataframe, 'shooting_by_zone', filename)
    print("saved: ", filename)
'''

saved:  regseas_shootzone_pergame_1996-97
saved:  regseas_shootzone_pergame_1997-98
saved:  regseas_shootzone_pergame_1998-99
saved:  regseas_shootzone_pergame_1999-00
saved:  regseas_shootzone_pergame_2000-01
saved:  regseas_shootzone_pergame_2001-02
saved:  regseas_shootzone_pergame_2002-03
saved:  regseas_shootzone_pergame_2003-04
saved:  regseas_shootzone_pergame_2004-05
saved:  regseas_shootzone_pergame_2005-06
saved:  regseas_shootzone_pergame_2006-07
saved:  regseas_shootzone_pergame_2007-08
saved:  regseas_shootzone_pergame_2008-09
saved:  regseas_shootzone_pergame_2009-10
saved:  regseas_shootzone_pergame_2010-11
saved:  regseas_shootzone_pergame_2011-12
saved:  regseas_shootzone_pergame_2012-13
saved:  regseas_shootzone_pergame_2013-14
saved:  regseas_shootzone_pergame_2014-15
saved:  regseas_shootzone_pergame_2015-16
saved:  regseas_shootzone_pergame_2016-17
saved:  regseas_shootzone_pergame_2017-18
saved:  regseas_shootzone_pergame_2018-19
saved:  regseas_shootzone_pergame_

'\nfor seas in seasons_list:\n    filename = "playoff_shootzone_pergame_{}".format(seas)\n    scrape_nba_season_data(shoot_by_zone_url, folder_name = \'shooting_by_zone\',                            metric_name = filename, season = seas, seas_type = \'Playoffs\')\n    time.sleep(1)\n    dataframe = parse_html(folder_name = \'shooting_by_zone\', file_name = filename, id_name="Crom_table__p1iZz",                            decompose_id=\'tr\', decompose_class="thead")\n    dataframe_to_csv(dataframe, \'shooting_by_zone\', filename)\n    print("saved: ", filename)\n'